## Escuela Superior de Cómputo del IPN.
### Matematicas Avanzadas para la Ciencia de Datos
Profesor: Alfonso
<br> 
Alumno: Huerta Ramírez  Michael Adolfo             

<center>Segundo parcial - 14 de Diciembre de 2022 </center>
<br>
Boleta: 2020602866

1. Considere el siguiente problema de programación lineal

<center><img src="./primero.png" width="500px"></center>

a) Construya el problema dual asociado

$ -2y_1 + 3y_2  $
<br>
$-y_1 - 2y_2 \geq 2$
<br>
$-2y_1 + y_2 \geq 3$
<br>
$-3y_1 - y_2 \geq 5$
<br>
$-y_1 + 3y_2 \geq 6$



b) Resuelva el problema anterior geométricamente

<center><img src="./grafica1.png" width="500px"></center>

##### Resultado ->
* Se observa que no es una región feasible por lo tanto no tiene solucion

c) Utilice información acerca del poblema lineal dual y los teoremas de
dualidad para resolver el problema primal

--> revisar

2. Resuelva el problema anterior utilizando una implementación propia del
método dual simplex


In [25]:
from dualsimplex import  SimplexMethod,MAX_MODE,MIN_MODE
import numpy as np

In [26]:
c1 = np.array([2,3,5,6])
a1 = np.array([
    [-1,-2,-3,-1],
    [-2,1,-1,3]
])
b1 = np.array([-2,-3]) 
a1, b1, c1 = make_dual(a1, b1, c1)  # cambiar a dual
simplex = SimplexMethod(c1, a1, b1, MIN_MODE)  # maximizar

print("Dual:")
simplex.print_task()
simplex.solve()

Dual:
-2.00y1 + -3.00y2 ->  MIN
y1 + 2.00y2 <= -2.0
2.00y1 + -y2 <= -3.0
3.00y1 + y2 <= -5.0
y1 + -3.00y2 <= -6.0

Iteration 0
     |   y1   |   y2   |   y3   |   y4   |   y5   |   y6   |    b   |
  y3 |   1.00 |   2.00 |   1.00 |   0.00 |   0.00 |   0.00 |  -2.00 |
  y4 |   2.00 |  -1.00 |   0.00 |   1.00 |   0.00 |   0.00 |  -3.00 |
  y5 |   3.00 |   1.00 |   0.00 |   0.00 |   1.00 |   0.00 |  -5.00 |
  y6 |   1.00 |  -3.00 |   0.00 |   0.00 |   0.00 |   1.00 |  -6.00 |
   F |   2.00 |   3.00 |   0.00 |   0.00 |   0.00 |   0.00 |  -0.00 |
   y |   0.00 |   0.00 |  -2.00 |  -3.00 |  -5.00 |  -6.00 |
ariable fue removida por: 4
     |   y1   |   y2   |   y3   |   y4   |   y5   |   y6   |    b   |
  y3 |   1.67 |   0.00 |   1.00 |   0.00 |   0.00 |   0.67 |  -6.00 |
  y4 |   1.67 |   0.00 |   0.00 |   1.00 |   0.00 |  -0.33 |  -1.00 |
  y5 |   3.33 |   0.00 |   0.00 |   0.00 |   1.00 |   0.33 |  -7.00 |
  y2 |  -0.33 |   1.00 |  -0.00 |  -0.00 |  -0.00 |  -0.33 |   2.00 |
   F |   3.00

False

3. Considere el siguiente programa lineal

    a) Escriba el problema dual y verifique que (4, 5) es una solucin factible.
    
    b) Use la información de la parte a) para derivar una solución  Óptima para los problemas dual y primal.

<center><img src="2.png" width="500px"></center>

a) Maximizar 

$19w_1 + 57w_2$
<br>
$w_1 + 2w_2 > 10$
<br>
$w_1 +4w_2 > 24$
<br>
$2w_1 + 3w_2 > 20$
<br>
$3w_1 + 2w_2 > 20$
<br>
$5w_1 + w_2 > 25$
<br>
$w_1,w_2,>0$

a.1)


4 + 2(5) = 14>10 ---> satisface la restriccion
<br>
4 + 4(5) = 24>24 ---> satisface la restriccion
<br>
2(4) + 3(5) = 23 > 20 ---> satisface la restriccion
<br>
3(4) + 2(5) = 22 > 20 ---> satisface la restriccion
<br>
5(4) + 5 = 25 > 25 
<br>
por lo tanto ( $w_1$ , $w_2$ ) = (4,5) es feasible para el dual

In [29]:
from dual_simplex import simplex , to_objective_function_value,dual_simplex_method

c3 = [10,24,20,20,25,0,0]
A3 = [
    [1,1,2,3,5,1,0],
    [2,4,3,2,1,0,1]
    ]
b3 = [19, 57]

primal3 = to_objective_function_value(c3, simplex(c3, A3, b3))
print('Primal: ', primal3)

Primal:  361.0


In [30]:
ca = [19,57,0,0,0,0,0]
Aa = [
    [-1,-2,1,0,0,0,0],
    [-1,-4,0,1,0,0,0],
    [-2,-3,0,0,1,0,0],
    [-3,-2,0,0,0,1,0],
    [-5,-1,0,0,0,0,1]   
]
ba = [-10,-24,-20,-20,-25]

dualb = to_objective_function_value(ca, dual_simplex_method(ca, Aa, ba))
print('Dual: ', dualb)

Dual:  361.0


4. Aplique el método primal dual al siguiente programa lineal

Maximizar $x_1 + 6x_2$

Sujeto a 

$x_1 + x_2 > 2$

$x_1 + 3x_2 < 3$

$x_1 , x_2 > 0 $

Convertir  >= a <= multiplicando ambos lados por (-1)

Maximizar $x_1 + 6x_2$

Sujeto a 

$-x_1 - x_2 < -2$

$x_1 + 3x_2 < -3$

$x_1 , x_2 > 0 $

En primal, hay 2 variables y 2 restricciones, por lo que en dual debe haber 2 restricciones y 2 variables

En primal, el coeficiente de la función objetivo $ c_1 = 1, c_2 = 6 $ se convierte en constantes del lado derecho en dual

En primal, las constantes del lado derecho $b_1 = -2, b_2 = -3 $ se convierten en coeficiente de función objetivo en dual

En primal, la función objetivo está maximizando, por lo que en la función objetivo dual debe estar minimizando

Sean $y1, y2$ las variables duales

$ -2 y_1 -3y_2 $

Sujeto a

$ y_1	+ y_2 ≥	1 $

$ y_1	+ 3y_2 ≥ 6 $

$ y_1, y_2 ≥ 0 $

5.- 

Considere el siguiente problema de programación lineal y su tabla óptima final como sigue.

<center><img src="captura_4.jpg" width="500px"></center>

a) Utilizando análisis de sensibilidad, encuentre la nueva solución óptima
si el coeficiente de x_2 en la función objetivo pasa de 1 a 5.

|       |       | $x_1$ | $x_2$ | $x_3$ | $x_4$ |$x_5$|$RHS$|          |
| ----- | ----- | ----- | ----- | ----- | ----- | --- | --- | -------- |
| z     | 1     | -5    | 3     | 2     | 0     | 16  |     |          |
| $x_1$ | 0     | 1     | 2     | 1     | 1     | 0   | 8   | $\frac{8}{2}$ = 4  |
| $x_5$ | 0     | 0     | -3    | -1    | 1     | 1   | 12  | $\frac{12}{3}$ = 4 |


|         |     | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $RHS$ |          
| ------- | --- | ----- | ----- | ----- | ----- | ----- | ----- | 
| 2 $x_1$ | 0   | 1     | 0     | $\frac{5}{3}$   | $\frac{1}{3}$   | $\frac{-2}{3}$  | $\frac{16}{3}$  |          
| 5 $x_2$ | 0   | 0     | 1     | -$\frac{1}{3}$  | $\frac{1}{3}$   | $\frac{1}{3}$   | 4     | 
| z ->    |     | 0     | 0     | $\frac{2}{3}$   | $\frac{7}{3}$   | $\frac{1}{3}$   | $\frac{92}{3}$  |




Por lo tanto para toda Z >= 0 obtenemos optimo

La solución es $\frac{92}{3}$